# Conversational RAG Agent
A conversational LLM app using LangChain & Chroma. Built from these tutorials: 
- [LangChain RAG](https://python.langchain.com/v0.2/docs/tutorials/rag/)
- [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/)

This exercise builds on the `conversatinal-rag-stateful.ipynb` exercise and adds in:
- A retrieval "agent" capable of manipulating steps of the process
- Tools the agent can access for stored document retrieval

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
%pip install -qU langchain-openai
%pip install python-dotenv
%pip install langchain_core
%pip install langgraph

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Enable tracing with LangSmith
# LANGCHAIN_API_KEY environment variable is set in .env
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = "conversational-rag-agent"

# Set the USER_AGENT environment variable
os.environ['USER_AGENT'] = 'conversational-rag-agent'

# 1. Load, chunk and index source documents to create a retriever.

In [2]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = WebBaseLoader(
    web_paths=(
        [
            "https://www.andyfitzgeraldconsulting.com/insights/what-is-information-architecture/",
            "https://www.andyfitzgeraldconsulting.com/insights/when-to-use-an-ia/",
            "https://www.andyfitzgeraldconsulting.com/insights/working-with-an-ia/",
            "https://www.andyfitzgeraldconsulting.com/insights/how-to-hire-an-ia/",
        ]
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("article"),    
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# 2. Convert retriever into a LangChain tool

In [3]:
from langchain.tools.retriever import create_retriever_tool

ia_info_tool = create_retriever_tool(
    retriever,
    "information_architecture_article_retriever",
    "Searches and returns excerpts from articles about information architecture.",
)
tools = [ia_info_tool]

# ia_info_tool.invoke("When does a project need an information architect?")

# 3. Create agent with conversation memory

In [28]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

prompt = """You are an assistant for question-answering tasks.
Use the tools at your disposal to answer the question. If you 
don't know the answer, say that you don't know. Use three 
sentences maximum and keep the answer concise."""

agent_executor = create_react_agent(llm, tools, state_modifier=prompt, checkpointer=memory)

# 4. Invoke the agent

In [ ]:
import uuid
from langchain_core.messages import HumanMessage

# create a new id each time this cell is run
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    question = input("> ")
    if question == "q":
        break
    response = agent_executor.invoke(
        {"messages": [HumanMessage(content=question)]}, config=config
    )
    print(response["messages"][-1].content, "\n")

# When does a project need information architecture work?
# What is information architecture?
# When should I hire an information architect?

In [ ]:
# inspect the response object
import json

print(json.dumps(response, indent=4, default=str))